### ДЗ2: Основные операторы PostgreSQL
Создаём таблицы:
```sql
CREATE TABLE customer (
    customer_id SERIAL PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    gender VARCHAR(20),
    DOB DATE,
    job_title VARCHAR(255),
    job_industry_category VARCHAR(100),
    wealth_segment VARCHAR(50),
    deceased_indicator CHAR(1),
    owns_car CHAR(3),
    address VARCHAR(255),
    postcode VARCHAR(10),
    state VARCHAR(100),
    country VARCHAR(100),
    property_valuation INTEGER
);

CREATE TABLE transaction (
    transaction_id SERIAL PRIMARY KEY,
    product_id INTEGER,
    customer_id INTEGER REFERENCES customer(customer_id),
    transaction_date DATE,
    online_order BOOLEAN,
    order_status VARCHAR(50),
    brand VARCHAR(100),
    product_line VARCHAR(100),
    product_class VARCHAR(50),
    product_size VARCHAR(50),
    list_price DECIMAL(10, 2),
    standard_cost DECIMAL(10, 2)
);
```


Заливаем csv через интерфейс DBeaver, он ругается на отсутствующий ключ.

![](https://raw.githubusercontent.com/Vendor62/MIPT_practice/refs/heads/main/homework/dsps_hw_2/media/01.png)

Удаляем руками из csv лишние строки и подключаемся к базе.

In [1]:
import psycopg2
import pandas as pd

In [2]:
host = "localhost" 
dbname = "postgres"
user = "postgres" 
password = "121233" 
port = 5432 

connection = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password,
        port=port
)

### 1. Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

In [3]:
cursor = connection.cursor()

cursor.execute("""
    SELECT DISTINCT(brand)
    FROM transaction
    WHERE standard_cost > 1500;
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,OHM Cycles
1,Trek Bicycles
2,Solex
3,Giant Bicycles


### 2. Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.

In [4]:
cursor.execute("""
    SELECT transaction_id
    FROM transaction
    WHERE order_status = 'Approved' 
        AND transaction_date BETWEEN '2017-04-01' AND '2017-04-09';
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,17
1,19
2,23
3,83
4,89
...,...
526,19655
527,19853
528,19899
529,19968


### 3.Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

In [5]:
cursor.execute("""
    SELECT DISTINCT(job_title)
    FROM customer
    WHERE (job_industry_category = 'IT' OR job_industry_category = 'Financial Services')
        AND job_title LIKE 'Senior%';
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,Senior Cost Accountant
1,Senior Developer
2,Senior Editor
3,Senior Financial Analyst
4,Senior Quality Engineer
5,Senior Sales Associate


### 4. Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services.

In [6]:
cursor.execute("""
    SELECT DISTINCT(brand)
    FROM transaction AS t
    JOIN customer AS c ON c.customer_id = t.customer_id
    WHERE job_industry_category = 'Financial Services';
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,
1,Trek Bicycles
2,WeareA2B
3,Solex
4,Giant Bicycles
5,OHM Cycles
6,Norco Bicycles


### 5. Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.

In [ ]:
cursor.execute("""
    SELECT DISTINCT(c.customer_id)
    FROM transaction AS t
    JOIN customer AS c ON c.customer_id = t.customer_id
    WHERE t.online_order = TRUE 
        AND t.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
    LIMIT 10;
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,1
1,2
2,3
3,5
4,6
5,7
6,8
7,11
8,12
9,13


### 6. Вывести всех клиентов, у которых нет транзакций.

In [8]:
cursor.execute("""
    SELECT c.customer_id
    FROM customer AS c
    LEFT JOIN transaction AS t ON c.customer_id = t.customer_id
    WHERE t.customer_id IS NULL;
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,852
1,869
2,1373
3,2074
4,2660
...,...
502,3996
503,3997
504,3998
505,3999


### 7. Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.

In [11]:
cursor.execute("""
    WITH max_cost AS (
        SELECT MAX(standard_cost) AS max_standard_cost
        FROM transaction
    )
    SELECT DISTINCT c.customer_id
    FROM customer AS c
    JOIN transaction AS t ON c.customer_id = t.customer_id
    JOIN max_cost AS m ON t.standard_cost = m.max_standard_cost
    WHERE c.job_industry_category = 'IT';
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,34
1,893
2,975
3,1672
4,1773
5,1918
6,2913
7,3151
8,3473


### 8. Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

In [12]:
cursor.execute("""
    SELECT DISTINCT c.customer_id
    FROM customer AS c
    JOIN transaction AS t ON c.customer_id = t.customer_id
    WHERE c.job_industry_category IN ('IT', 'Health')
        AND t.order_status = 'Approved'
        AND t.transaction_date BETWEEN '2017-07-07' AND '2017-07-17';
""")

display(pd.DataFrame(cursor.fetchall()))

,0
0,22
1,28
2,41
3,47
4,104
...,...
110,3255
111,3288
112,3360
113,3365


In [15]:
cursor.execute("ROLLBACK")